In [ ]:
!pip install openai
!pip install PyPDF2
!pip install python-docx
!pip install pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184487 sha256=e5db3d8e8dad88dadbcb8c8b371a9e326d1b95d7479829cd27c1db535ea81603
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 22.5 MB/s eta 0:00:00


In [ ]:
import openai
import PyPDF2
import docx
import os
import re
import pandas as pd
from pdfminer.high_level import extract_text

In [ ]:
def get_all_resumes(folder_path):
    resumes = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            text = convert_files_to_text(file_path)
            resumes.append(text)
    return resumes

def convert_files_to_text(file_path):
    if file_path.endswith('.pdf'):
        text = convert_pdf_to_text2(file_path)
    elif file_path.endswith('.docx'):
        text = convert_docx_to_text(file_path)
    else:
        return ""
    return text

def convert_pdf_to_text2(file_path):
    text = extract_text(file_path)
    return text

def convert_pdf_to_text(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
    return text

def convert_docx_to_text(file_path):
    doc = docx.Document(file_path)
    text = ''
    for paragraph in doc.paragraphs:
        text += paragraph.text + '\n'
    return text

def get_choice_text_from_prompt(messages):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0,
            max_tokens=1000
        )
        choice_text = response.choices[0]["message"]["content"]
        return choice_text
    except Exception as e:
        print("Error in get_choice_text_from_prompt:", str(e))
        return ""

def analyze_resume(resumetext,job_description,skills):
    system_message = """
    You are an excellent talent recuiter and your task is to score resumes of candidates between 0-100 against a job description and skills required.
    You will be provided with candidate resume, job description and skills required for the job.

    The system instruction is:
    Step-1: First check whether the candidate's resume is an actual resume or not.
    Step-2: If the candidate's resume is not an actual resume then score=0, else further
    analyse the candidate's resume against the job description and skills required by looking for these following qualities:
      1. Relevant Experience: Relevant work experience in the field or industry related to the job role
      2. Duration of experiences
      3. Previous job titles
      4. Specific responsibilities and their impact
      5. Achievements in previous experiences
      6. Education - The candidate's educational background
      7. Educational quality
      8. Certifications: specialized training, especially if they align with the job requirements.
    Step-3:
    Score the overall quality of resume against the job description and skills required between 0-100.
    Score should be such that it can be compared against different candidate's resumes for shortlisting purpose.
    Score should be a floating point number with upto 2 decimal point accuracy.
    Step-4:
    Return  the final score of resume, answer should be in json format with keys as - score and detailed explanation of the scoring procedure.
    """
    user_message = f"""
    Score the resume of candidate out of 100 against the given job description and skill requirements.
    Information about the candidate's resume, skills/requirements and job description are given inside text delimited by triple backticks.

    Candidate's Resume :```{resumetext}```
    The skills/requirements expected from the candidates: ```{skills}```
    Job Description for the Target Role: ```{job_description}```
    """
    messages =  [
                {'role':'system',
                'content': system_message},
                {'role':'user',
                'content': user_message},
                ]

    resume_score = get_choice_text_from_prompt(messages)
    return resume_score

def analyze_all_resumes(folder_path, job_description, skills):
  resumes = get_all_resumes(folder_path)
  scores = []
  for filename, resume_text in zip(os.listdir(folder_path), resumes):
      resume_score = analyze_resume(resume_text, job_description, skills)
      scores.append((filename, resume_score))

  df = pd.DataFrame(scores, columns=["File_Name", "Score"])

  return df

In [ ]:
api_key = ''
openai.api_key = api_key

In [ ]:
job_description = input("Enter the job description:\n")

Enter the job description:
General Responsibilities :    Data Collection and Validation: Collaborate with stakeholders to understand their data requirements and gather relevant information from various sources. Validate the accuracy and completeness of the data collected Report Generation: Develop and maintain standardized and ad-hoc reports to assist stakeholders in understanding key findings, risks, and opportunities Data Analysis: Utilize data analysis techniques to uncover insights, identify anomalies, and recognize patterns within the data to identify fraud Process Improvement: Continuously identify opportunities to streamline data gathering processes and improve efficiency Cross-functional Collaboration: Work closely with teams, such as finance, compliance, and engineering, to access relevant data availability and ensure consistency in reporting Compliance and Governance: Ensure adherence to data privacy regulations and internal data governance policies when handling sensitive in

In [ ]:
skills = input("Enter the skills/requirements for the job:\n")

Enter the skills/requirements for the job:
Analytical Skills, SQL, Tableau, and Python (Programming Language),Microsoft Excel, Communication, Data Analysis, Problem Solving, Microsoft Office, Microsoft PowerPoint, and Business Analysis


In [ ]:
folder_path = input("Enter the folderpath:\n")

Enter the folderpath:
/content/drive/MyDrive/Resume


In [ ]:
df = analyze_all_resumes(folder_path, job_description, skills)
print(df)

              File_Name                                              Score
0     Final_CV_Data.pdf  {\n  "score": 72.5,\n  "explanation": "The can...
1  Resume_Baglari-1.pdf  To score the candidate's resume against the jo...
2            Aditya.pdf  {\n  "score": 62.5,\n  "explanation": "The can...
3          ashutosh.pdf  {\n  "score": 62.5,\n  "explanation": "The can...
4             Ayush.pdf  {\n  "score": 72.5,\n  "explanation": "The can...
5            Naveen.pdf  {\n  "score": 62.5,\n  "explanation": "The can...
6         Rohan sur.pdf  {\n  "score": 75.5,\n  "explanation": "The can...
7            Sakshi.pdf  {\n  "score": 72.5,\n  "explanation": "The can...
8            Shubhi.pdf  {\n  "score": 65.75,\n  "explanation": "The ca...


In [ ]:
print(df['Score'])

0    None
1    None
Name: Score, dtype: object


In [ ]:
# Initialize lists to store scores and explanations
import json
data=df
scores = []
explanations = []

# Iterate through the Series and extract scores and explanations
for item in data:
    try:
        json_data = json.loads(item)
        scores.append(json_data["score"])
        explanations.append(json_data["explanation"])
    except json.JSONDecodeError:
        scores.append(None)
        explanations.append(None)

# Create a DataFrame
df = pd.DataFrame({"Score": scores, "Explanation": explanations})

# Print the DataFrame
print(df)

  Score Explanation
0  None        None
1  None        None


# Single File Test

In [ ]:
filepath='/content/drive/MyDrive/Scanned docs/X registration.pdf'
resume_text = convert_files_to_text(filepath)
system = """
    You are an excellent talent recuiter and your task is to score resumes between 0-100 against a job description and skills required.
    You will be provided with candidate resume, job description and skills required for the job.

    The system instruction is:
    Step-1: First check whether the candidate's resume is an actual resume or not.
    Step-2: If the candidate's resume is not an actual resume then score=0, else further
    analyse the candidate's resume by looking for these following qualities:
      1. Relevant Experience: Relevant work experience in the field or industry related to the job role,
      focus on the duration of employment, job titles, and the specific responsibilities and achievements listed under each role.
      2. Education and Certifications: The candidate's educational background and any relevant certifications
      or specialized training are essential, especially if they align with the job requirements.
      3. Key Skills: Specific skills mentioned, these can include technical skills, soft skills, language proficiency,
      and any other abilities that are critical for the role.
      4. Achievements and Accomplishments: Candidate's abilities and contributions in previous roles.
      5. Adaptability and Learning Ability: Candidates who can quickly learn and adapt to new technologies, processes, or changes in the workplace.
      6. Extracurricular and Volunteer Activities: Involvement in community work or extracurricular activities that showcase leadership or teamwork skills.
      7. Professionalism and Formatting: A well-organized, error-free, and professional resume.
    Step-3:
    Score the overall quality of resume against the job description and skills required between 0-100.
    Step-4:
    Return only the final score of resume, answer should be in json format with key as - score.
    """
prompt = f"""
    Information about the candidate's resume, skills/requirements and job description are given inside text delimited by triple backticks.

    Candidate's Resume :```{resume_text}```

    The skills/requirements expected from the candidates: ```{skills}```

    Job Description for the Target Role: ```{job_description}```
    """
messages =  [
{'role':'system', 'content':system},
{'role':'user', 'content': prompt}]

resume_score = get_choice_text_from_prompt(messages)
print(resume_score)

{
  "score": 0
}


In [ ]:
      # 9. Technical skills
      # 10. Soft skills
      # 11. Language proficiency
      # 12. Awards, Achievements and Accomplishments
      # 13. Adaptability and Learning Ability: Candidates who can quickly learn and adapt to new technologies, processes, or changes in the workplace.
      # 14. Extracurricular and Volunteer Activities: Involvement in community work or extracurricular activities that showcase leadership or teamwork skills.
      # 15. Professionalism and Formatting: A well-organized, error-free, and professional resume.